In [1]:
%load_ext autoreload
%autoreload 2

In [28]:
import sys

sys.path.insert(0, '../')
from src.mongo_utils import collection, create_set
from src.utils import sanitize_context
import regex as re

In [3]:
def get_all_entries():
    query = {"$and": [{"beispiele": {"$exists": True}}, {"bedeutungen": {"$exists": True}}]}

    # Retrieving documents that match the query
    return collection.find(query)

In [4]:
test = get_all_entries()
test = list(test)

In [37]:
def evaluate_sets(sets):
    def compare_words(words):
        # Check if all words in the list are the same
        if len(words) < 2:
            return True  # If there's only one word or none, they are considered "equal"
        first_word = words[0]
        return all(word == first_word for word in words)
    
    complain_data = []
    for title, context_word, context_sentence, gt in sets:
        complain = False
        if context_word and re.match("[^\p{L}]", context_word):
            if title != context_word:
                complain = True
        extracted_context = re.findall(r"''(.*?)''", context_sentence)
        extracted_context = [sanitize_context(i) for i in extracted_context]
        a = []
        for x in extracted_context:
            if not re.match(r"\[\[(.*?)\]\]", x):
                a.append(x.strip())
        extracted_context = a
        if len(extracted_context) > 1 and context_word is not None:
            if not compare_words(extracted_context) and context_word != extracted_context[0]:
                complain = True
            
        if complain:
            complain_data.append(
                [title, context_word, context_sentence, gt]
            )
        
    return complain_data

In [ ]:
from pprint import pprint

complaints = []
for a in test[:]:
    complaints += evaluate_sets(create_set(a))
pprint(complaints)
print(len(complaints))

In [41]:
len(complaints)

282

In [6]:
create_set(a)

[('Sonntag',
  'Sonntag',
  "Neuere Arbeitsgesetze lassen für den ''Sonntag'' wieder mehr und mehr Ausnahmen zu.",
  'Wochentag zwischen Samstag und Montag'),
 ('Sonntag',
  'Sonntag',
  "„Es ist ''Sonntag'' gewesen, und nun ist es Montag geworden, auch in Altholm.“",
  'Wochentag zwischen Samstag und Montag'),
 ('Sonntag',
  'Sonntag',
  "„In den Abstimmungen vom ''Sonntag'' unterstützen alle drei Standortgemeinden dieses Alternativkonzept und erteilen dem Projekt des Bundes für einen Business-Flugplatz eine Absage.“",
  'Wochentag zwischen Samstag und Montag'),
 ('Sonntag',
  'Sonntag',
  "„Zu gefährlichen Jagdszenen in der Luft kam es am ''Sonntag'' über Oberammergau. Ein Gleitschirmpilot befand sich im Anflug auf einen Oberammergauer Landeplatz, als ihn das ferngesteuerte Flugobjekt fast zum Absturz brachte.“",
  'Wochentag zwischen Samstag und Montag'),
 ('Sonntag',
  'Sonntag',
  "„In Frankreich wurden Berichten zufolge am ''Sonntag'' die Leichen von drei Kajakfahrern gefunden.“"